# Prime Numbers with Python

### Sidclay da Silva

### April 2022
---
### Introduction
This notebook contains a *Python* code to generate a list of prime numbers with the mathematical background used to improve performance.

---

### Definition
1. An integer $p \in \mathbb{Z}$, $p \neq 1$, is a prime number in $\mathbb{Z}$ when it can only be divided by $1$ or $p$.
1. The set of prime numbers is infinite - Euclid's theorem, a fundamental statement in number theory.
1. If $c \in \mathbb{Z}$, $c \neq 1$, is not a prime number it is called composite number, and it can be written as a product of prime numbers, $c= p\,k$.
1. If $c=p\,k$, then $c\div p=k$ and $c\div k=p$.

The opposite number of a prime is also a prime number, but we will stick to the positive ones without any loss to this work.

---

### Development

From the definition (1) above, checking if an integer $n$ is a prime number is quite straight, we only need to check if it is a multiple of any integer $i \in \big[2,n\big)$, if it isn't, then $n$ is a prime number. 

A integer $n$ is multiple of $i$ when the operation $n \div i$ results in a integer, or in other words the operation leaves remainder zero. Let's define a function to check it for the given $n$.

In [1]:
def is_prime(n=2):
    """Returns True if 'n' is a prime number"""
    n = int(abs(n))
    r = range(2,n)
    p = True
    for i in r:
        if n % i == 0: 
            p = False
            break
    
    return p

Now a quick test using an short interval containing some well known primes - $\big[2,11\big]$.

In [2]:
[(i,is_prime(i)) for i in range(2,12)]

[(2, True),
 (3, True),
 (4, False),
 (5, True),
 (6, False),
 (7, True),
 (8, False),
 (9, False),
 (10, False),
 (11, True)]

For checking results of higher magnitude, the *Wikipedia* page https://en.wikipedia.org/wiki/List_of_prime_numbers can be used, they have the list of the first one thousand primes.

Next we test it with a larger list, up to $100000$, the interval $\big[2,1e5\big]$, and check how long does it take to run. The resulting list is going to be huge, then we count the primes to keep it simple.

The execution time depends on the equipment being used, here it is used to compare perfomance between versions.

In [3]:
import time

t0 = time.time()

plist = [is_prime(i) for i in list(range(2,int(1e5+1)))]

t1 = time.time()
print('Count:', sum(plist), '\nExecution:', round(t1-t0,3),'s')

Count: 9592 
Execution: 61.756 s


The function to check if a number is prime works, but for checking a huge list of numbers its performance does not impress anyone.

Let's try a different approach, instead of testing unique integers, for a give $n$, our function should return the list of primes in the the interval $\big[2,n\big]$.

According to definition (1), prime numbers are greater than one, if $n<2$ we simply return an empty list.

The definition (3) can help us speed things up, let's dive into it.

If $c_1=p\, k_1$ and $c_2=c_1\, x$,

then $c_2=(p\, k_1)x = p\, k_2$.

Meaning that we only need to test if the elements in the list $\big[2,n\big]$ are multiples of prime numbers, and not, as did our initial function, testing them for each integer less than or equal to $n$, e.g. to check if $11$ is prime, we test if it is multiple of $\big\{2,3,5,7\big\}$ only.

The new function builds up a list of primes, which is initiated as \[2\], then it tests if $3$ is multiple of any element in the list, at this point only the $2$, if it isn't, then $3$ is added to the primes list \[2,3\]. Next it tests $4$ in the same way, and so on.

In [4]:
def prime_list(n=2):
    "Returns the list of prime numbers 'p <= n'"
    if n < 2: return []

    primes = [2]
    i = 3
    
    while i <= n:
        p = True
        for prime in primes:
            # here we test only with primes
            if i % prime == 0:
                p = False
                break

        if p == True: primes.append(i)

        i += 1

    return primes

In [5]:
t0 = time.time()

plist = prime_list(int(1e5))

t1 = time.time()
print('Count:', len(plist), '\nLast:', plist[-1], '\nExecution:', round(t1-t0,3),'s')

Count: 9592 
Last: 99991 
Execution: 5.493 s


Much better than previous performance, but we can improve it more.

First, using definition (3) again. We already know that $2$ is prime and all even numbers can be written as $n=2k$, that means they are all composite numbers multiples of $2$. We will just skip them during the iteration, this will remove half of the elements in the list.

Second we use definition (4), to recall it, if $c=p\,k$, then $c\div p=k$ and $c\div k=p$.

An example, the divisors of $16$ are $\big\{1,2,4,8,16\big\}$, when $16$ is divided by any of its divisors the result is also one of of its divisors.
- $16\div 1 = 16$
- $16\div 2 = 8$
- $16\div 4 = 4$
- $16\div 8 = 2$
- $16\div 16 = 1$

Nothing new here, but according to the defnition (4) we only would need to divide $16$ by $\big\{1,2,4\big\}$ to find all of its divisors, this is interesting, right?

Let's see $24$ and its divisors $\big\{1,2,3,4,6,8,12,24\big\}$, this time we would divide $24$ by $\big\{1,2,3,4\big\}$ to find all of its divisors. If you think about a number like $1e5$, then the definition (4) turns out to be a huge time saving resource.

What is even more interesting is that $\sqrt{16}=4$ and $\sqrt{24}\cong 4.90$. It is not a coincidence that for $16$ and $24$ we could divide up to $4$. One way to illustrate it is using geometry. For example, which are the possible different rectangles with area $16\,\textrm{mm}^2$, considering only integers? The answer, $1\times 16$, $2\times 8$ and $4\times 4$, the last one is the square of side $\sqrt{16}=4$, after that we start to repeat the rectangles.

Back to our prime numbers, to check if an integer $n$ is prime, it is enough to test if it is multiple of the prime numbers $p\leq \sqrt{n}$, this is a time saving resource as well.

Great, let's enhance our function with these two additional resources and see if it really gets faster.

In [6]:
def prime_list(n=2):
    "Returns the list of prime numbers 'p <= n'"
    if n < 2: return []

    primes = [2]
    i = 3
    
    while i <= n:
        p = True
        for prime in primes:
            # here we stop testing using the square root
            if prime > i**.5: break

            if i % prime == 0:
                p = False
                break

        if p == True: primes.append(i)

        # here we skip even numbers
        i += 2

    return primes

In [7]:
t0 = time.time()

plist = prime_list(int(1e5))

t1 = time.time()
print('Count:', len(plist), '\nLast:', plist[-1], '\nExecution:', round(t1-t0,3),'s')

Count: 9592 
Last: 99991 
Execution: 0.227 s


This was a good impromevent in performance, right?

So far we have built a list of prime numbers in the interval $\big[2,n\big]$, but you could change it to find the list of the first $n$ primes, only changing the *while* test to check the length of the *primes* list - "*while len(primes) <= n:*".

This first approach was done by aggregation, building up the primes list, and with some mathematical resources we could improve performance. Next we try a different approach, by exclusion let's say, we start with the whole list of integers, then we remove all the composite numbers from it

How does that work?

We initiate the list using the interval $\big[2,n\big]$ and set the primes list to $[]$. We start with $2$, removing all its multiples ($n=2k$) from the original list and then we include the $2$ itself into the primes list.

After that we take the new first element of the list, which should be $3$, and do the same, remove all its multiples ($n=3k$) from the list, include it to the primes list, and take the new first element, $5$, because $4$ was removed as multiple of $2$.

As we previously saw, it is enough to do it when $n\leq \sqrt{n}$, after that should not remain any composite number to be removed. This is great, don't you think?

Let's see an example with the interval $\big[2,30\big]$, keeping in mind that $\sqrt{30}=5,48$

| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|(list)|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|

First we remove the multiples of $2$ and include it in our primes list.

| | | | | | | | | | | | | | | |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|(list)|3|5|7|9|11|13|15|17|19|21|23|25|27|29|

| | |
|:-:|:-:|
|(primes)|2|

Now we take $3$, remove its multiples and update the primes list with it.

| | | | | | | | | | |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|(list)|5|7|11|13|17|19|23|25|29|

| | | |
|:-:|:-:|:-:|
|(primes)|2|3|

The same with $5$.

| | | | | | | | |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|(list)|7|11|13|17|19|23|29|

| | | | |
|:-:|:-:|:-:|:-:|
|(primes)|2|3|5|

At this point all the elements left in the list are prime numbers, the composite ones are gone. To finish the job, we only need to put all of them in one list.

| | | | | | | | | | |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|2|3|5|7|11|13|17|19|23|29|

In our function the list is already initiated without even numbers greater than $2$.

In [8]:
def prime_list(n=2):
    "Returns the list of prime numbers 'p <= n'"
    if n < 2: return []
    
    nlist = [2] + list(range(3, n+1, 2))
    limit = int(n**.5)
    primes = []
    prime = nlist[0]
    
    while prime <= limit:
        # the mask contains True for multiples of 'prime'
        mask = [n % prime == 0 for n in nlist]

        # rebuild the list with elements where mask is not True
        nlist = [n for n,m in zip(nlist,mask) if m != True]

        # include 'prime' to primes list
        primes.append(prime)

        # take the new first element
        prime = nlist[0]

    # merge the primes list with elements left from original list
    primes.extend(nlist)
    return primes

In [9]:
t0 = time.time()

plist = prime_list(int(1e5))

t1 = time.time()
print('Count:', len(plist), '\nLast:', plist[-1], '\nExecution:', round(t1-t0,3),'s')

Count: 9592 
Last: 99991 
Execution: 0.247 s


Actually, it was quite the same performance as the previous one, but we could try it using *Numpy* and see how it performs.

*Numpy* uses arrays instead of lists to manipulate data, it holds powerfull mathematical resources, it requires less memory and its processing is faster than using lists, these differences make enormous advantage when dealing with large amount of data.

*Numpy* also has easier sintaxe, e.g. the mask we made for the list previously is included directly as array selection criteria in a single line of code.

Let's change the function and see how it goes, the logic behind the code is the same.

In [10]:
import numpy as np

def prime_list(n=2):
    "Returns the list of prime numbers 'p <= n'"
    if n < 2: return []
    
    narray = np.append([2], np.arange(start=3, stop=n+1, step=2))
    limit = int(np.sqrt(n))
    primes = []
    prime = narray[0]
    
    while prime <= limit:
        primes.append(prime)
        # only one line of code to rebuild the list
        narray = narray[narray % prime !=0]
        prime = narray[0]

    return list(np.append(primes, narray))

In [11]:
t0 = time.time()

plist = prime_list(int(1e5))

t1 = time.time()
print('Count:', len(plist), '\nLast:', plist[-1], '\nExecution:', round(t1-t0,3),'s')

Count: 9592 
Last: 99991 
Execution: 0.034 s


This was really good improvement!

You could try *Numpy* in the first version, the aggregation approach, and see what happens.

---

### Conclusion

In this notebook we have developed a function using *Python* to generate prime numbers. Its performance could be improved based on mathematical background, which was explained along with the code.